In [92]:
import random
from math import floor
from scipy.sparse import csc_matrix, eye, find
import numpy as np

In [100]:
class RANDU:
    def __init__(self, mul, mod,seed):
        self.mul = mul
        self.mod = mod
        self.currvalue = seed
        self.seed = seed
    def rand(self):
        self.currvalue = (self.currvalue*self.mul) % self.mod
        return self.currvalue
    def rewind(self):
        self.currvalue = self.seed

def matrixpower( M, p):
    M1 = M.copy()
    for _ in range(p-1):
        M1 = M1.dot(M)
    return M1.copy()
class Generator:
    def __init__(self, j, k, seed):
        self.j = j
        self.k = k
        self.g = RANDU(65539, 2**31, seed)
        self.currvalue = [self.g.rand() for _ in range(k)]
        self.initial = self.currvalue
        self.seed = seed #список начальных значений
        self.currpos = 0
    def rand(self):
        self.currvalue[self.currpos] = self.currvalue[self.currpos] + self.currvalue[(self.currpos-self.j) % self.k]
        val = self.currvalue[self.currpos]
        self.currpos = (self.currpos+1)%self.k
        return val
    def rewind(self):
        self.currvalue = self.initial
        self.currpos = 0
    
    def jumpahead(self, jlen):
        A = csc_matrix(([1]*(self.k+1), ([i for i in range(self.k-1)]+[self.k-1]*2, [i for i in range(1,self.k)]+[0, self.k-self.j])))
        m, r = divmod(jlen, self.k)
        B, C = 0, 0
        dif =  self.k - self.j - r
        if dif>0:
            B = matrixpower(A, r)
            C = B.copy()
            C.dot(matrixpower(A, dif))
        else:
            C = matrixpower(A, self.k - self.j)
            B = C.copy()
            B.dot(matrixpower(A, -dif))
        C = C + eye(self.k)
        C = matrixpower(C, m)
        #for i, j in find(C):
        #    C[i,j] = C[i,j] % 2
        return (B.dot(C)).dot(np.array(self.currvalue))[0]

In [98]:
seed = 1
gen = Generator(24, 55, seed)

662650498
889043850
1517105810
1099405258
1556661794
1532516010
1658779090
347529642
209626306
678846154
1528339154
797728906
1466507682
854895658
1882716050
1158399658
97497090
1156067338
1107447826
236237002


In [34]:
gen.rewind()

In [3]:
A = csc_matrix(([1]*10, ([0,1,2,3,4,5, 6,7,8,8], [1,2,3,4,5,6,7, 8, 0,4])))


In [5]:
A.indices


array([8, 0, 1, 2, 3, 8, 4, 5, 6, 7], dtype=int32)

In [6]:
A.todense()

matrix([[0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int32)

In [99]:
print(gen.jumpahead(1000))

1134404389984662.0


In [95]:
for _ in range(1000):
    gen.rand()

In [96]:
gen.currvalue[gen.currpos]

71672560229907016